# Module for Cleaning the Data

## Importing Libraries

In [1]:
# Python Libraries
import pandas as pd
import numpy as np
import os

# Suppress warnings to clean output
import warnings
warnings.filterwarnings("ignore")

## Local Functions

This function count how many times the selected color appeared in a selected period (hour/day/month).

In [2]:
def frequency(color_name) -> list:
    """
        (function)
        color_name = string
        This function count how many times the selected color appeared in a selected period.
    """

    # True/False array for the selected color
    where_color_appeared = df['Cor'] == color_name

    # How many times that color appeared (reset every hour)
    color_per_hour = where_color_appeared.cumsum()-where_color_appeared.cumsum().where(reset_time).ffill().fillna(0).astype(int)
    
    # How many colors (total) apperead (reset every hour)
    total_colors_array = pd.Series([True for x in range(df.shape[0])])
    total_colors_array = total_colors_array.cumsum()-total_colors_array.cumsum().where(reset_time).ffill().fillna(0).astype(int)
    total_colors_array = np.where(total_colors_array == 0, total_colors_array + 1, total_colors_array) # Adjust for 0 values = +1 -> Error: Can not divide by 0

    return color_per_hour / total_colors_array

## Get the original data

In [3]:
def get_data():
    import os

    path = '../Double_data/original data'
    df = pd.DataFrame()

    for path_ in os.listdir(path):
        df_ = pd.read_excel(path +'/'+ path_)
        df = pd.concat([df, df_], axis=0, ignore_index=True)

    return df.drop_duplicates()

df = get_data()
df.to_csv('../Double_data/AUG_clear.csv', index=False)

## Load and clean the data

In [4]:
# Load the dataset
path = '../Double_data/AUG_clear.csv'
df = pd.read_csv(path)

# Split String Time datatype to Minute and Hour integer
df['Hour'] = df['Minuto'].str[:2].astype(int)
df['Minute'] = df['Minuto'].str[-2:].astype(int)

# Split String Time datatype to Day/Month/Year integer
df['Day'] = df['Data'].str[5:7].astype(int)
df['Month'] = df['Data'].str[8:10].astype(int)
df['Year'] = df['Data'].str[-4:].astype(int)

# Transform String Currency datatype to float
df['Red bet'] = df['Apostas no Vermelho'].str.replace('.', '').str.replace(',', '.').str[3:].astype(float)
df['White bet'] = df['Apostas no Branco'].str.replace('.', '').str.replace(',', '.').str[3:].astype(float)
df['Black bet'] = df['Apostas no Preto'].str.replace('.', '').str.replace(',', '.').str[3:].astype(float)

# Drop old columns and sort the columns by Day, Hour and Minute
df.drop(['Data', 'Minuto', 'Apostas no Vermelho', 'Apostas no Branco', 'Apostas no Preto'], axis=1, inplace=True)
df.sort_values(by=['Day', 'Hour', 'Minute'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a new feature: Balance
df['Balance'] = 0

# Black and Red Bet prize is twice the input ammount
# White Bet prize is 14x the input ammount
df.loc[(df['Número'] >= 8), 'Balance'] = df['Red bet'] + df['White bet'] + df['Black bet'] - 2 * df['Black bet']
df.loc[(df['Número'] >= 1)&(df['Número'] <= 7), 'Balance'] = df['White bet'] + df['Black bet'] + df['Red bet'] - 2 * df['Red bet']
df.loc[(df['Número'] == 0), 'Balance'] = df['Red bet'] + df['Black bet'] + df['White bet'] - 14 * df['White bet']

# Create a new feature: Wallet
# This feature will show the ammount of money that Blaze has won during an hour (or day)
df['Wallet'] = 0
reset_time = np.where((df['Minute'] == 0) & (df['Minute'].shift(1).fillna(1) != 0) & (df['Day'] != df['Day'].shift(1).fillna(0)), True, False) # This auxiliar array select where to reset the Wallet
df['Wallet'] = df['Balance'].cumsum()-df['Balance'].cumsum().where(reset_time).ffill().fillna(1).astype(float)

# Create a new feature:
# How many Black/Red/White colors appears whitin an hour (in percentage)
reset_time = np.where((df['Minute'] == 0) & (df['Minute'].shift(1).fillna(1) != 0), True, False) # This auxiliar array select where to reset the Wallet
for color in df.Cor.unique():
    df[f'{color}_per_hour'] = 0
    df[f'{color}_per_hour'] = frequency(color)

# Change 'Cor' column to integers and columns name
colors = {
    'White':0,
    'Red':1,
    'Black':2
}
df['Cor'] = df['Cor'].map(colors)
df.rename(columns={'Número':'Number', 'Cor':'Color'}, inplace=True)

# 'Seed' column is useless in this context
df.drop('Seed', axis=1, inplace=True)

df[100:110]

,Number,Color,Hour,Minute,Day,Month,Year,Red bet,White bet,Black bet,Balance,Wallet,Red_per_hour,Black_per_hour,White_per_hour
100,9,2,0,50,1,1,2023,565.66,3678.68,575.33,3669.01,-137727.86,0.480000,0.380000,0.140000
101,0,0,0,50,1,1,2023,522.88,2802.89,549.87,-35364.82,-173092.68,0.475248,0.376238,0.148515
102,12,2,0,51,1,1,2023,1376.53,2074.80,651.12,2800.21,-170292.47,0.470588,0.382353,0.147059
103,9,2,0,51,1,1,2023,905.34,2165.69,743.21,2327.82,-167964.65,0.466019,0.388350,0.145631
104,9,2,0,52,1,1,2023,2047.74,2366.46,980.50,3433.70,-164530.95,0.461538,0.394231,0.144231
105,12,2,0,52,1,1,2023,1279.58,2046.16,875.31,2450.43,-162080.52,0.457143,0.400000,0.142857
106,6,1,0,53,1,1,2023,5543.65,1906.26,1064.80,-2572.59,-164653.11,0.462264,0.396226,0.141509
107,14,2,0,53,1,1,2023,3303.41,2048.23,883.99,4467.65,-160185.46,0.457944,0.401869,0.140187
108,11,2,0,54,1,1,2023,1054.64,1702.05,693.39,2063.30,-158122.16,0.453704,0.407407,0.138889
109,14,2,0,54,1,1,2023,929.63,2014.19,763.30,2180.52,-155941.64,0.449541,0.412844,0.137615


In [5]:
df.sort_values(by=['Day', 'Hour', 'Minute']).to_csv('../Double_data/AUG_clear.csv', index=False)